In [2]:
! python -m pip install -U ipykernel ipywidgets jupyter IPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.0/812.0 kB 3.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: IPython
    Found existing installation: ipython 8.21.0
    Uninstalling ipython-8.21.0:
      Successfully uninstalled ipython-8.21.0

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import spacy
import pandas as pd
import os


"""
os.chdir("..")


nlp = spacy.load("en_core_web_trf")
"""

'\nos.chdir("..")\n\n\nnlp = spacy.load("en_core_web_trf")\n'

In [2]:
df = pd.read_pickle("data/rsc_dnb.pkl", compression="zip")

In [3]:
from tqdm import tqdm

tqdm.pandas()

In [4]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=6)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [6]:
# get the unique texts
def get_nlp(row):
    bio = row["bio"]
    if bio is not None:
        try:
            return nlp(bio)
        except Exception as e:
            print(bio)
            print(e)
            return None
    else:
        return None
    
df_texts = pd.DataFrame(df["bio"].unique(), columns=["bio"])

df_texts["doc"] = df_texts.progress_apply(get_nlp, axis=1)

 48%|████▊     | 510/1052 [04:48<03:20,  2.70it/s]

nan
[E1041] Expected a string, Doc, or bytes as input, but got: <class 'float'>


100%|██████████| 1052/1052 [09:30<00:00,  1.84it/s]


In [ ]:
df_texts

In [7]:
# map the texts back to bio with the doc
df_texts = df_texts.set_index("bio")

df["doc"] = df["bio"].map(df_texts["doc"])

In [8]:
df

,id,issn,title,fpage,lpage,year,volume,journal,author,type,...,primaryTopic,primaryTopicPercentage,secondaryTopic,secondaryTopicPercentage,text,modified_text,authors,resolved,bio,doc
0,rspa_1909_0086,0950-1207,Harmonic tidal constants for certain Chinese a...,127,130,1909,83,Proceedings of the Royal Society of London. Se...,"Thomas Wright|Sir George H. Darwin, K. C. B., ...",article,...,Meteorology,45.186419,Tables,18.209093,"Harmonic Tidal Constants , etc. In the table i...","Harmonic Tidal Constants , etc. In the table i...",Thomas Wright,"Wright, Thomas (fl.1740-1760)","WRIGHT , THOMAS ( fl . 1740–1760), author of ...","( , WRIGHT, ,, THOMAS, (, fl, ., 1740–1760, ),..."
1,rspa_1920_0042,0950-1207,Obituary notices of fellows deceased.,0,0,1920,97,Proceedings of the Royal Society of London. Se...,William Watson,article,...,Biography,69.344447,Meteorology,6.751422,OBITUARY NOTICES FELLOWS DECEASED .\nVOL. XCVI...,OBITUARY NOTICES FELLOWS DECEASED . VOL. XCVII...,William Watson,"Watson, William (1715-1787)","WATSON , Sir WILLIAM (1715–1787), physician, ...","( , WATSON, ,, Sir, WILLIAM, (, 1715–1787, ), ..."
2,rspb_1914_0004,0950-1193,The alleged excretion of creatine in carbohydr...,205,220,1914,87,Proceedings of the Royal Society of London. Se...,"George Graham|E. P. Poulton|Dr. F. G. Hopkins,...",article,...,Biochemistry,76.408311,Chemistry 2,15.401245,205 The Alleged Excretion of Creatine in Carbo...,205 The Alleged Excretion of Creatine in Carbo...,George Graham,"Graham, George (1675-1751)","GRAHAM , GEORGE (1675–1751), mechanician, was...","( , GRAHAM, ,, GEORGE, (, 1675–1751, ), ,, mec..."
3,rspl_1870_0006,0370-1662,XVI. On linear differential equations. \#x2014...,14,17,1870,19,Proceedings of the Royal Society of London,"W. H. L. Russell, F. R. S.",abstract,...,Formulae,71.760816,Tables,26.947171,]\gt ; 14 Mr. W. H. L. Russell on Linear Diffe...,]> ; 14 Mr. W. H. L. Russell on Linear Differe...,"W. H. L. Russell, F. R. S.","Russell, William (1822-1892)","RUSSELL , Sir WILLIAM (1822–1892), lieutenant...","( , RUSSELL, ,, Sir, WILLIAM, (, 1822–1892, ),..."
4,rspl_1870_0011,0370-1662,XXI. Researches in animal electricity.,22,28,1870,19,Proceedings of the Royal Society of London,"C. B. Radcliffe, M. D.|Charles Brooke, M. A.",abstract,...,Electricity,48.020748,Nervous System,35.838707,"22 Dr , C. B. RadclifFe on Animal Electricity ...","22 Dr , C. B. RadclifFe on Animal Electricity ...","Charles Brooke, M. A.","Brooke, Charles (1804-1879)","BROOKE , CHARLES (1804–1879), surgeon and inv...","( , BROOKE, ,, CHARLES, (, 1804–1879, ), ,, su..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6277,41141783,02610523,A comparison of the diurnal changes of intensi...,177,187,1826,116,Philosophical Transactions of the Royal Societ...,Henry Foster,fla,...,Meteorology,43.051230,Astronomy,30.676141,VII .\nA comparison of the diurnal changes oj ...,VII . A comparison of the diurnal changes oj i...,Henry Foster,"Foster, Henry","FOSTER , HENRY (1796–1831), navigator, born i...","( , FOSTER, ,, HENRY, (, 1796–1831, ), ,, navi..."
6278,41141784,02610523,Account of the repetition of Mr. Christie&apos...,188,205,1826,116,Philosophical Transactions of the Royal Societ...,S. H. CHRISTIE|Henry Foster,fla,...,Measurement,32.441429,Meteorology,26.965145,VIII .\nAccount of the repetition of Mr. Chris...,VIII . Account of the repetition of Mr. Christ...,Henry Foster,"Foster, Henry","FOSTER , HENRY (1796–1831), navigator, born i...","( , FOSTER, ,, HENRY, (, 1796–1831, ), ,, navi..."
6279,41141785,02610523,Observations to determine the amount of Atmosp...,206,230,1826,116,Philosophical Transactions of the Royal Societ...,W. E. Parry|Henry Foster|J. C. Ross,fla,...,Astronomy,61.861393,Meteorology,19.819962,IX .\nObservations to determine the amount of ...,IX . Observations to determine the amount of A...,Henry Foster,"Foster, Henry","FOSTER , HENRY (1796–1831), navigator, born i...","( , FOSTER, ,, HENRY, (, 1796

In [12]:
# drop the none types
df = df.dropna(subset=["doc"]).reset_index(drop=True)

In [13]:
# get (ent, label) pairs for each sentence like this
# [[(ent.text, ent.label_) for ent in doc.ents] for sent in doc.sents]

df["ents"] = df["doc"].progress_apply(
    lambda doc: [[(ent.text, ent.label_) for ent in sent.ents] for sent in doc.sents]
)

100%|██████████| 6270/6270 [01:59<00:00, 52.49it/s] 


In [17]:
df.iloc[1]["ents"]

[[],
 [('WATSON', 'PERSON'),
  ('WILLIAM', 'PERSON'),
  ('3 April 1715', 'DATE'),
  ("St. John's Street", 'FAC'),
  ('Smithfield', 'GPE'),
  ('London', 'GPE')],
 [("the Merchant Taylors' school", 'ORG'),
  ('1726', 'DATE'),
  ('1730', 'DATE'),
  ('Richardson', 'PERSON')],
 [('annually', 'DATE'), ("the Apothecaries' Company", 'ORG')],
 [('1738', 'DATE'), ('Watson', 'PERSON')],
 [('9 April 1741', 'DATE')],
 [('the ‘Philosophical Transactions', 'WORK_OF_ART'),
  ('more than fifty-eight', 'CARDINAL')],
 [('Watson', 'PERSON'),
  ('the Royal Society', 'ORG'),
  ('Thursdays', 'DATE'),
  ('first', 'ORDINAL'),
  ('the Mitre in Fleet Street', 'FAC'),
  ('the Crown and Anchor in the Strand', 'FAC'),
  ('Pulteney', 'PERSON'),
  ('ii', 'CARDINAL'),
  ('333', 'CARDINAL')],
 [('1745', 'DATE'),
  ('Hans Sloane', 'PERSON'),
  ('Godfrey Copley', 'PERSON'),
  ('Copley', 'PERSON')],
 [('Sloane', 'PERSON'),
  ('the British Museum', 'ORG'),
  ('Montagu House', 'FAC'),
  ('1756', 'DATE'),
  ('Watson', 'PERSO

In [18]:
unique_bios

,bio
0,"WRIGHT , THOMAS ( fl . 1740–1760), author of ..."
1,"WATSON , Sir WILLIAM (1715–1787), physician, ..."
2,"GRAHAM , GEORGE (1675–1751), mechanician, was..."
3,"RUSSELL , Sir WILLIAM (1822–1892), lieutenant..."
4,"BROOKE , CHARLES (1804–1879), surgeon and inv..."
...,...
1045,"HODGKINSON , GEORGE CHRISTOPHER (1816–1880), ..."
1046,"LAMBE, WILLIAM (1766–1847), physician, son of..."
1047,"LEARED , ARTHUR, M.D. (1822–1879), traveller,..."
1048,"PROCTOR , RICHARD ANTHONY (1837–1888), astron..."


In [19]:
unique_bios = pd.DataFrame(df["bio"].unique(), columns=["bio"])

# limit the bios to the first six sentences
def limit_sents(row, limit=6):
    bio = row["bio"]
    if bio is not None:
        return " ".join([sent.text for sent in nlp(bio).sents][:limit])
    else:
        return None

unique_bios["short_bio"] = unique_bios.progress_apply(limit_sents, axis=1)
unique_bios.to_csv("data/unique_bios.csv", index=False)

100%|██████████| 1050/1050 [16:33<00:00,  1.06it/s]   


In [37]:
sample = unique_bios.sample(200).reset_index(drop=True)

In [38]:
# get ents from df using bio as key
def get_docs(row):
    bio = row["bio"]
    doc = df[df["bio"] == bio]["doc"].values
    return doc

sample["doc"] = sample.progress_apply(get_docs, axis=1)


100%|██████████| 200/200 [00:00<00:00, 2997.29it/s]


In [46]:
sample.rename(columns={"bio": "text"}, inplace=True)

sample = pd.DataFrame(sample["text"].values, columns=["text"])

In [47]:
sample

,text
0,"CURTIS , Sir ROGER (1746–1816), admiral, was ..."
1,"EAMES , JOHN ( d . 1744), dissenting tutor, w..."
2,"PULTENEY , RICHARD (1730–1801), botanist, bor..."
3,"GOODRICKE , JOHN (1764–1786), astronomer, bor..."
4,"MUSGRAVE , SAMUEL (1732–1780), physician and ..."
...,...
195,"BOYS , THOMAS (1792–1880), theologian and ant..."
196,"WALKER , RICHARD (1679–1764), professor of mo..."
197,"MURPHY , ROBERT (1806–1843), mathematician, b..."
198,"COBBOLD , THOMAS SPENCER , M.D. (1828–1886), ..."


In [40]:
# use the doc to get works of art with their offsets
def get_ents(row):
    doc = row["doc"]
    ents = [[(ent.text, ent.label_, ent.start_char, ent.end_char) for ent in sent.ents if ent.label_ == "WORK_OF_ART"] for sent in doc]
    #flatten the list
    ents = [ent for sublist in ents for ent in sublist]
    return ents

sample["predictions"] = sample.progress_apply(get_ents, axis=1)

100%|██████████| 200/200 [00:00<00:00, 1445.82it/s]


In [1]:
sample

NameError: name 'sample' is not defined

In [48]:
sample.to_csv("data/sample_bios.csv", index=False)

In [5]:
unique_bios = pd.read_csv("../data/unique_bios.csv")

In [6]:
unique_bios

,bio,short_bio
0,"WRIGHT , THOMAS ( fl . 1740–1760), author of ...","WRIGHT , THOMAS ( fl . 1740–1760), author of ..."
1,"WATSON , Sir WILLIAM (1715–1787), physician, ...","WATSON , Sir WILLIAM (1715–1787), physician,..."
2,"GRAHAM , GEORGE (1675–1751), mechanician, was...","GRAHAM , GEORGE (1675–1751), mechanician, wa..."
3,"RUSSELL , Sir WILLIAM (1822–1892), lieutenant...","RUSSELL , Sir WILLIAM (1822–1892), lieutenan..."
4,"BROOKE , CHARLES (1804–1879), surgeon and inv...","BROOKE , CHARLES (1804–1879), surgeon and inv..."
...,...,...
1045,"HODGKINSON , GEORGE CHRISTOPHER (1816–1880), ...","HODGKINSON , GEORGE CHRISTOPHER (1816–1880),..."
1046,"LAMBE, WILLIAM (1766–1847), physician, son of...","LAMBE, WILLIAM (1766–1847), physician, son o..."
1047,"LEARED , ARTHUR, M.D. (1822–1879), traveller,...","LEARED , ARTHUR, M.D. (1822–1879), traveller..."
1048,"PROCTOR , RICHARD ANTHONY (1837–1888), astron...","PROCTOR , RICHARD ANTHONY (1837–1888), astron..."


In [7]:
training_data = pd.read_csv("dnb_100.csv")

In [8]:
training_data

,annotation_id,annotator,created_at,id,label,lead_time,text,updated_at
0,5,1,2024-02-14T12:34:17.531622Z,1,"[{""start"":1,""end"":19,""text"":""CURTIS , Sir ROGE...",284.711,"CURTIS , Sir ROGER (1746–1816), admiral, was ...",2024-02-23T13:48:34.346590Z
1,6,1,2024-02-14T12:34:39.149019Z,2,"[{""start"":1,""end"":13,""text"":""EAMES , JOHN"",""la...",257.107,"EAMES , JOHN ( d . 1744), dissenting tutor, w...",2024-02-23T13:47:18.864157Z
2,7,1,2024-02-14T12:34:51.783283Z,3,"[{""start"":1,""end"":19,""text"":""PULTENEY , RICHAR...",729.033,"PULTENEY , RICHARD (1730–1801), botanist, bor...",2024-02-23T13:52:16.837070Z
3,8,1,2024-02-14T12:35:15.570275Z,4,"[{""start"":1,""end"":17,""text"":""GOODRICKE , JOHN""...",2246.620,"GOODRICKE , JOHN (1764–1786), astronomer, bor...",2024-02-23T13:51:51.528999Z
4,9,1,2024-02-14T12:35:27.705398Z,5,"[{""start"":1,""end"":18,""text"":""MUSGRAVE , SAMUEL...",423.493,"MUSGRAVE , SAMUEL (1732–1780), physician and ...",2024-02-23T13:52:42.539615Z
...,...,...,...,...,...,...,...,...
95,103,1,2024-02-25T20:41:49.864433Z,101,"[{""start"":1,""end"":22,""text"":""MOLYNEUX , Sir TH...",225.368,"MOLYNEUX , Sir THOMAS (1661–1733), physician,...",2024-02-25T20:41:49.864445Z
96,104,1,2024-02-25T20:46:12.537460Z,102,"[{""start"":1,""end"":20,""text"":""CHESELDEN , WILLI...",251.718,"CHESELDEN , WILLIAM (1688–1752), surgeon and ...",2024-02-25T20:46:12.537472Z
97,105,1,2024-02-25T20:48:03.216242Z,103,"[{""start"":1,""end"":14,""text"":""REEVE , HENRY"",""l...",86.071,"REEVE , HENRY (1780–1814), physician, was sec...",2024-02-25T20:48:03.216253Z
98,106,1,2024-02-25T20:48:31.173981Z,104,"[{""start"":1,""end"":21,""text"":""BROOKE , HENRY JA...",25.689,"BROOKE , HENRY JAMES (1771–1857), crystallogr...",2024-02-25T20:48:31.173992Z


In [9]:
# get 100 new bios from unique bios if they are not already in the training data
new_bios = unique_bios[~unique_bios["bio"].isin(training_data["text"])].sample(100)

In [10]:
new_bios

,bio,short_bio
454,"CRUSIUS , LEWIS (1701–1775), biographer, was ...","CRUSIUS , LEWIS (1701–1775), biographer, was..."
1006,"HERAPATH , WILLIAM (1796–1868), analytical ch...","HERAPATH , WILLIAM (1796–1868), analytical c..."
71,"MAITLAND , CHARLES (1815–1866), author, born ...","MAITLAND , CHARLES (1815–1866), author, born..."
159,"DALE , SAMUEL (1659?–1739), physician, son of...","DALE , SAMUEL (1659?–1739), physician, son of..."
800,"PROUT , WILLIAM (1785–1850), physician and ch...","PROUT , WILLIAM (1785–1850), physician and c..."
...,...,...
310,"CAMPBELL , GEORGE (1719–1796), divine, was bo...","CAMPBELL , GEORGE (1719–1796), divine, was bo..."
501,"MONTAGU, WILLIAM (1720?–1757), captain in the...","MONTAGU, WILLIAM (1720?–1757), captain in the..."
505,"CAVENDISH , CHARLES (1620–1643), royalist gen...","CAVENDISH , CHARLES (1620–1643), royalist gen..."
107,"KING , Sir EDMUND (1629–1709), physician, bor...","KING , Sir EDMUND (1629–1709), physician, bo..."


In [11]:
new_bios = pd.DataFrame(new_bios["bio"].values, columns=["text"])
new_bios = new_bios.sample(100).reset_index(drop=True)

new_bios

,text
0,"BANKS , Sir JOSEPH (1743–1820), president of ..."
1,"FOX , WILSON (1831–1887), physician, son of a..."
2,"BAGFORD , JOHN (1650–1716), 'shoemaker and bi..."
3,"HEBERDEN , WILLIAM , the elder (1710–1801), p..."
4,"BURNEY , JAMES (1750–1821), captain in the ro..."
...,...
95,"CHARLETT , ARTHUR (1655–1722), master of Univ..."
96,"WILKINSON , JOHN (1728–1808), ‘father of the ..."
97,"NEVE , TIMOTHY (1694–1757), divine and antiqu..."
98,"LYSONS , DANIEL, M.D. (1727–1800), physician,..."


In [13]:
# save as csv
new_bios.to_csv("new_bios.csv", index=False)